In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(416))
random.seed(40)
random.shuffle(index)

train_index = index[0:291]
val_index = index[291:333]
test_index = index[333:416]

In [3]:
#dataimage_train = [dataimage[j] for j in train_index]
#print(dataimage_train)

In [4]:
lab = np.zeros((416,1)) #first AD:188 then CN:288
lab[0:188, 0] = 1 #AD -- 1; CN -- 0
print(lab[2].shape)

(1,)


In [5]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_AD_CN_Jan25.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((416,1)) #first AD:188 then CN:288
        self.lab[0:188, 0] = 1 #AD -- 1; CN -- 0
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[train_index]
            self.lab = self.lab[train_index]
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
       
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 

        return sample


In [6]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=400,
                        shuffle=True, num_workers=4)

print("finished")

great job dude
great job dude
great job dude
finished


In [7]:
import torch.nn as nn
import torch.nn.functional as F

#initial: 
#121 * 121 * 3(feature map number)
#60 * 60 * 16
#30 * 30 * 32
#15 * 15 * 64
#7 * 7 * 128
#3 * 3 * 256

#Linear: 3 * 3 * 256
#3 * 3 * 256(2304)--->1024 --->256 ---> 64 --->2 
class Net_2D(nn.Module):
    def __init__(self):
        super(Net_2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3 * 3 * 256, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 3 * 3 * 256)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net_2d = Net_2D().cuda()

print("finished")

finished


In [8]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


def alexnet(pretrained=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = AlexNet(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['alexnet']))
        for p in model.features.parameters():
            p.requires_grad = False

        # fine-tune the last convolution layer
        for p in model.features[10].parameters():
            p.requires_grad = True

    model.classifier.add_module('fc_out', nn.Linear(1000,2))
    model.classifier.add_module('sigmoid', nn.LogSoftmax())

    stdv = 1.0 / math.sqrt(1000)
    for p in model.classifier.fc_out.parameters():
        p.data.uniform_(-stdv, stdv)

    return model

In [9]:
import torch.nn as nn
import torch.nn.functional as F
class Light_Net(nn.Module):
    def __init__(self):
        super(Light_Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 8, 5, padding=2)
        self.conv5 = nn.Conv3d(8, 8, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(8 * 8 * 8 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 8 * 8 * 8 * 5)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

#net = Light_Net().cuda()
print("finished")

finished


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_2d.parameters(), lr=0.005, momentum=0.9)

print("finished")

finished


In [ ]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

epoch = 50000
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_2d.train()

        # forward + backward + optimize
        outputs = net_2d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_2d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_2d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
        
    if(total_val_loss < min_total_loss):
        min_total_loss = total_val_loss
   
    #save model
    #best_model
    torch.save(net_2d, 'best_model_Jan25_2d_2types.pkl')

print("we're done training")

In [16]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_Jan25_2d_2types.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("\n", results_type, " ", labels)

    
for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        correct_sum += 1
    total_sum += 1

correct_sum_AD = 0
correct_sum_CN = 0
#correct_sum_MCI = 0

AD_num = 0
#MCI_num = 0
CN_num = 0
for label in labels:
    if (label.float() == 1):
        AD_num += 1
    if (label.float() == 0):
        CN_num += 1
    #if (label.float() == 2):
        #MCI_num += 1
print("labels", AD_num, CN_num)
#print("correct_sum_final", correct_sum)
#print(total_sum)
AD_num = 0
#MCI_num = 0
CN_num = 0
for results in results_type:
    if (results.float() == 1):
        AD_num += 1
    if (results.float() == 0):
        CN_num += 1
    #if (results.float() == 2):
        #MCI_num += 1
#print("predicted", AD_num, CN_num, MCI_num)


for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        if(results_type[i].float() == 1):
            correct_sum_AD += 1
    if(labels[i].float() == results_type[i].float()):
        if(results_type[i].float() == 0):
            correct_sum_CN += 1
    #if(labels[i].float() == results_type[i].float()):
        #if(results_type[i].float() == 2):
            #correct_sum_MCI += 1
    total_sum += 1
    
print("correct_sum", correct_sum)
print("correct_sum_AD", correct_sum_AD)
print("correct_sum_CN", correct_sum_CN)
#print("correct_sum_MCI", correct_sum_MCI)
print(total_sum)

0
tensor([[ 0.0661, -0.0617],
        [ 0.0660, -0.0617],
        [ 0.0661, -0.0617],
        [ 0.0662, -0.0618],
        [ 0.0663, -0.0618],
        [ 0.0662, -0.0617],
        [ 0.0661, -0.0618],
        [ 0.0661, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0661, -0.0618],
        [ 0.0661, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0661, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0661, -0.0618],
        [ 0.0662, -0.0618],
        [ 0.0661, -0.0618],
        [ 0.0662, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0661, -0.0618],
        [ 0.0662, -0.0618],
        [ 0.0661, -0.0617],
        [ 0.0661, -0.0617],
        [ 0.0661, -0.0617],
        [ 0.0661, -0.0618],
        [ 0.0662, -0.0617],
        [ 0.0662, -0.0617],
        [ 0.0662, -0.0618],
        [ 0.0662, -0.0618],
        [ 0.0661, -0.0618],
        [ 0.0661, -0.0618],
        [ 0.0661, -0.0617],
        [ 0.0661, -0.0617],
        [ 0.0662, 